In [1]:
import argparse
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import os
import anndata
from scipy.io import mmread

In [2]:
def load_anndata(mtx_dir):
    # load in mtx, barcodes and features
    matrix_path = os.path.join(mtx_dir, 'matrix.mtx.gz')
    matrix = mmread(matrix_path).tocsr()
    barcodes_path = os.path.join(mtx_dir, 'barcodes.tsv.gz')
    barcodes = pd.read_csv(barcodes_path,sep='\t',compression='gzip',header=0,
                          names = ['barcodes'])
    features_path = os.path.join(mtx_dir, 'features.tsv.gz')
    features = pd.read_csv(features_path, sep='\t',compression='gzip',header=0, 
                           names=['ensemblID','name', 'feature_type', 'chr', 'start', 'end'])
    
    # select gene features
    gene_features = np.where('feature_type' == 'Gene Expression')[0]
    
    # generate anndata
    var = features.iloc[gene_features]
    var.index = var['name']
    obs = barcodes
    obs.index = obs['barcodes']
    rna_adata = anndata.AnnData(matrix[:,gene_features],
                    var = var,
                    obs = obs)

    return rna_adata

In [3]:
mtx_dir = '../data/Sample_BCG_h1/raw_feature_bc_matrix'

In [4]:
matrix_path = os.path.join(mtx_dir, 'matrix.mtx.gz')
matrix = mmread(matrix_path).tocsr()
barcodes_path = os.path.join(mtx_dir, 'barcodes.tsv.gz')
barcodes = pd.read_csv(barcodes_path,sep='\t',compression='gzip',header=None,
                      names = ['og_barcode'])
features_path = os.path.join(mtx_dir, 'features.tsv.gz')
features = pd.read_csv(features_path, sep='\t',compression='gzip',header=None, 
                       names=['ensemblID','name', 'feature_type', 'chr', 'start', 'end'])

In [38]:
barcodes = pd.read_csv(barcodes_path,sep='\t',compression='gzip',header=None,
                      names = ['og_barcode'])

In [39]:
features = pd.read_csv(features_path, sep='\t',compression='gzip',header=None, 
                       names=['ensemblID','name', 'feature_type', 'chr', 'start', 'end'])

In [40]:
barcodes.shape

(693857, 1)

In [41]:
features.shape

(122348, 6)

In [42]:
matrix.shape

(122348, 693857)

In [35]:
gene_features = np.where(features['feature_type'] == 'Gene Expression')[0]

In [62]:
var = features.iloc[gene_features]
var.index = var['name'].tolist()
obs = barcodes
obs.index = obs['og_barcode'].tolist()
rna_adata = anndata.AnnData(matrix[gene_features,:].T,
                var = var,
                obs = obs)

/home/paddockl/miniconda3/envs/multiome_winner/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [45]:
rna_adata

AnnData object with n_obs × n_vars = 693857 × 36601
    obs: 'og_barcode'
    var: 'ensemblID', 'name', 'feature_type', 'chr', 'start', 'end'

In [48]:
rna_adata.var

,ensemblID,name,feature_type,chr,start,end
MIR1302-2HG,ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267
FAM138A,ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
OR4F5,ENSG00000186092,OR4F5,Gene Expression,chr1,65418,69055
AL627309.1,ENSG00000238009,AL627309.1,Gene Expression,chr1,120931,133723
AL627309.3,ENSG00000239945,AL627309.3,Gene Expression,chr1,91104,91105
...,...,...,...,...,...,...
AC141272.1,ENSG00000277836,AC141272.1,Gene Expression,KI270728.1,1270983,1270984
AC023491.2,ENSG00000278633,AC023491.2,Gene Expression,KI270731.1,13000,13001
AC007325.1,ENSG00000276017,AC007325.1,Gene Expression,KI270734.1,72410,72411
AC007325.4,ENSG00000278817,AC007325.4,Gene Expression,KI270734.1,131493,131494


In [22]:
var

,ensemblID,name,feature_type,chr,start,end
FAM138A,ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
OR4F5,ENSG00000186092,OR4F5,Gene Expression,chr1,65418,69055
AL627309.1,ENSG00000238009,AL627309.1,Gene Expression,chr1,120931,133723
AL627309.3,ENSG00000239945,AL627309.3,Gene Expression,chr1,91104,91105
AL627309.2,ENSG00000239906,AL627309.2,Gene Expression,chr1,140338,140339
...,...,...,...,...,...,...
AC141272.1,ENSG00000277836,AC141272.1,Gene Expression,KI270728.1,1270983,1270984
AC023491.2,ENSG00000278633,AC023491.2,Gene Expression,KI270731.1,13000,13001
AC007325.1,ENSG00000276017,AC007325.1,Gene Expression,KI270734.1,72410,72411
AC007325.4,ENSG00000278817,AC007325.4,Gene Expression,KI270734.1,131493,131494


In [23]:
obs

,og_barcode
0,AAACAGCCAAACATAG-1
1,AAACAGCCAAACCCTA-1
2,AAACAGCCAAACCTAT-1
3,AAACAGCCAAACCTTG-1
4,AAACAGCCAAACGCGA-1
...,...
693851,TTTGTTGGTTTGGGTA-1
693852,TTTGTTGGTTTGGTTC-1
693853,TTTGTTGGTTTGTCTA-1
693854,TTTGTTGGTTTGTGGA-1


In [25]:
matrix[gene_features,:]

<36600x693857 sparse matrix of type '<class 'numpy.int64'>'
	with 18470196 stored elements in Compressed Sparse Row format>

In [26]:
matrix

<122348x693857 sparse matrix of type '<class 'numpy.int64'>'
	with 75641395 stored elements in Compressed Sparse Row format>

In [27]:
matrix[:10,:10].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [49]:
ad = sc.read_h5ad('../results/Sample_BCG_h3/pre_filt_rna_adata.h5ad')

In [50]:
ad

AnnData object with n_obs × n_vars = 716250 × 36601
    obs: 'og_barcode', 'batch', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'ensemblID', 'name', 'feature_type', 'chr', 'start', 'end', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [51]:
rna_adata

AnnData object with n_obs × n_vars = 693857 × 36601
    obs: 'og_barcode'
    var: 'ensemblID', 'name', 'feature_type', 'chr', 'start', 'end'

In [52]:
RIBO_GENE_PATH = '/data/peer/lpaddock/data/utils/RB_genes_human'

In [63]:
adata = rna_adata

In [73]:
var = features.iloc[gene_features]
var.index = var['name'].tolist()
obs = barcodes
obs.index = obs['og_barcode'].tolist()
rna_adata = anndata.AnnData(matrix[gene_features,:].T,
                var = var,
                obs = obs)

/home/paddockl/miniconda3/envs/multiome_winner/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [74]:
# calculate qc metrics
adata.var_names_make_unique()
sc.pp.calculate_qc_metrics(adata, inplace = True)
# ribo content
ribo_genes = pd.read_csv(RIBO_GENE_PATH, header = None)[0].tolist()
ribo_genes = adata.var_names[adata.var_names.isin(ribo_genes)]
adata.var['ribo'] = adata.var_names.isin(ribo_genes)
row_sum_adata_ribo = np.sum(adata[:, ribo_genes].X.toarray(), axis=1)
adata.obs['ribo_pct'] = row_sum_adata_ribo / adata.obs['total_counts'] * 100
# mito content
mt_genes = adata.var_names[adata.var_names.str.startswith('MT-')]
adata.var['mito'] = adata.var_names.isin(mt_genes)
row_sum_adata_mito = np.sum(adata[:, mt_genes].X.toarray(), axis=1)
adata.obs['mt_pct'] = row_sum_adata_mito / adata.obs['total_counts'] * 100


In [77]:
adata.var['ribo'].unique()

array([False,  True])

In [58]:
any(adata.var_names.isin(ribo_genes))

True

In [59]:
adata.var_names[adata.var_names.isin(ribo_genes)]

Index(['MRPL20', 'RPL22', 'RPL11', 'MRPS15', 'RPS8', 'MRPL37', 'RPL5',
       'MRPS21', 'MRPL9', 'RPS27',
       ...
       'RPL3', 'RPS19BP1', 'PRPS2', 'RPS4X', 'RPL36A', 'PRPS1', 'RPL39',
       'RPL10', 'RPS4Y1', 'RPS4Y2'],
      dtype='object', length=173)

In [4]:
adata = sc.read_h5ad('../results/Sample_BCG_h1/pre_filt_rna_adata.h5ad')

In [5]:
adata

AnnData object with n_obs × n_vars = 693857 × 36601
    obs: 'og_barcode', 'batch', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'ribo_pct', 'mito_pct'
    var: 'ensemblID', 'name', 'feature_type', 'chr', 'start', 'end', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'ribo', 'mito'

In [8]:
adata.var

,ensemblID,name,feature_type,chr,start,end,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,ribo,mito
MIR1302-2HG,ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267,0,0.000000,0.000000,100.000000,0,0.000000,False,False
FAM138A,ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081,0,0.000000,0.000000,100.000000,0,0.000000,False,False
OR4F5,ENSG00000186092,OR4F5,Gene Expression,chr1,65418,69055,0,0.000000,0.000000,100.000000,0,0.000000,False,False
AL627309.1,ENSG00000238009,AL627309.1,Gene Expression,chr1,120931,133723,19,0.000027,0.000027,99.997262,19,2.995732,False,False
AL627309.3,ENSG00000239945,AL627309.3,Gene Expression,chr1,91104,91105,1,0.000001,0.000001,99.999856,1,0.693147,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC141272.1,ENSG00000277836,AC141272.1,Gene Expression,KI270728.1,1270983,1270984,0,0.000000,0.000000,100.000000,0,0.000000,False,False
AC023491.2,ENSG00000278633,AC023491.2,Gene Expression,KI270731.1,13000,13001,1,0.000001,0.000001,99.999856,1,0.693147,False,False
AC007325.1,ENSG00000276017,AC007325.1,Gene Expression,KI270734.1,72410,72411,0,0.000000,0.000000,100.000000,0,0.000000,False,False
AC007325.4,ENSG00000278817,AC007325.4,Gene Expression,KI270734.1,131493,131494,24,0.000035,0.000035,99.996541,24,3.218876,False,False
